In [ ]:
%%bash
git clone https://github.com/PKU-YuanGroup/Video-LLaVA
cd Video-LLaVA
conda create -n videollava python=3.10 -y
conda activate videollava
pip install --upgrade pip  # enable PEP 660 support
pip install -e .
pip install -e ".[train]"
pip install flash-attn --no-build-isolation
pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/Video-LLaVA
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finis

In [ ]:
%%bash
pip install -U transformers
python -m pip install av

Cloning into 'Video-LLaVA'...
bash: line 3: conda: command not found
bash: line 4: conda: command not found
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.0.1 which is incompatible.
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-req-build-if6cnq2p
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
videollava 1.0.0 requires tokenizers<0.14,>=0.12.1, but you have tokenizers 0.19.1 which is incompatible.
videollava 1.0.0 requires transformers==4.31.0, but you have transformers 4.42.4 which is incompatible.


In [ ]:
!pip install --upgrade transformers
!pip install --upgrade videollava

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
import sys
import os

# Añadir el directorio raíz al sys.path
sys.path.append('/content/Video-LLaVA')

import torch
from videollava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from videollava.conversation import conv_templates, SeparatorStyle
from videollava.model.builder import load_pretrained_model
from videollava.utils import disable_torch_init
from videollava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria


def predict(query, path_video):
    disable_torch_init()
    # video = '/content/Video-LLaVA/videollava/serve/examples/sample_demo_1.mp4'
    video = path_video
    inp = query
    model_path = 'LanguageBind/Video-LLaVA-7B'
    cache_dir = 'cache_dir'
    device = 'cuda'
    #device = 'cpu'
    load_4bit, load_8bit = True, False
    model_name = get_model_name_from_path(model_path)
    tokenizer, model, processor, _ = load_pretrained_model(model_path, None, model_name, load_8bit, load_4bit, device=device, cache_dir=cache_dir)
    video_processor = processor['video']
    conv_mode = "llava_v1"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles

    video_tensor = video_processor(video, return_tensors='pt')['pixel_values']
    if type(video_tensor) is list:
        tensor = [video.to(model.device, dtype=torch.float16) for video in video_tensor]
    else:
        tensor = video_tensor.to(model.device, dtype=torch.float16)

    print(f"{roles[1]}: {inp}")
    inp = ' '.join([DEFAULT_IMAGE_TOKEN] * model.get_video_tower().config.num_frames) + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=tensor,
            do_sample=True,
            temperature=0.1,
            max_new_tokens=1024,
            use_cache=True,
            stopping_criteria=[stopping_criteria])

    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    #print(outputs)
    return outputs

#if __name__ == '__main__':
#   main()

In [ ]:
import pandas as pd

In [ ]:
path_test_models = '/content/shareddrive/MyDrive/Colab_Notebooks/POLITO/Machine learning and deep learning/Project/Test_models/'
path_root_videos = path_test_models + 'segmented_videos/predictions/low_resolution/'
path_sample_50_videos_with_pred = path_test_models + 'sample_50_videos_with_pred.csv'

In [ ]:
sample_50_videos_with_pred = pd.read_csv(path_sample_50_videos_with_pred).set_index("id")

In [ ]:
sample_50_videos_with_pred['Answer_Video-LLaVA'] = ''

In [ ]:
sample_50_videos_with_pred.head(2)

,Unnamed: 0,scenario,video_uid,clip_uid,annotation_uid,query_idx,clip_start_sec,clip_end_sec,mIoU,query,Answer,clip_start_sec_pred,clip_end_sec_pred,Answer_Video-LLaVA
id,,,,,,,,,,,,,,
1,0,Carpenter\nFixing something in the home\nTalki...,fffbaeef-577f-45f0-baa9-f10cabf62dfb,07cc6e65-df8c-46fb-b4be-b808d777454f,efa2bedf-3eb9-43c3-9581-da4a64922820,1,87.98,126.94,0.89,In what location did I see the laptop?,on the desk,85.405,125.860004,
2,1,Car mechanic,805989f6-0696-4de2-ad9b-0f194e0ac48d,eaf8d34a-0e20-45d0-a288-569df047461e,450c2f2b-5172-4d6f-91fa-495c7f1fb986,6,454.67,480.03,0.78,How many drawers did I open?,8,454.500,475.500000,


In [ ]:
name_dev = input("Type your name: ")

Type your name: mateo


In [ ]:
for video_id, query in zip(sample_50_videos_with_pred.index, sample_50_videos_with_pred['query']):
    if name_dev.lower() == 'mateo' and video_id <= 25:
        # Store the prediction result in a variable
        prediction_result = predict(query, path_root_videos + str(video_id) + '.mp4')
        # Assign the result to the 'Answer_Video-LLaVA' column for the current video_id
        sample_50_videos_with_pred.loc[video_id, 'Answer_Video-LLaVA'] = prediction_result
        if video_id == 25:
            break
    elif video_id > 25:
        # Store the prediction result in a variable
        prediction_result = predict(query, path_root_videos + str(video_id) + '.mp4')
        # Assign the result to the 'Answer_Video-LLaVA' column for the current video_id
        sample_50_videos_with_pred.loc[video_id, 'Answer_Video-LLaVA'] = prediction_result


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at LanguageBind/Video-LLaVA-7B were not used when initializing LlavaLlamaForCausalLM: ['model.video_tower.video_tower.encoder.layers.14.self_attn.out_proj.weight', 'model.video_tower.video_tower.encoder.layers.23.self_attn.q_proj.weight', 'model.video_tower.video_tower.encoder.layers.7.self_attn.q_proj.bias', 'model.video_tower.video_tower.encoder.layers.7.self_attn.out_proj.weight', 'model.image_tower.image_tower.encoder.layers.19.self_attn.k_proj.weight', 'model.image_tower.image_tower.encoder.layers.21.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.12.mlp.fc1.bias', 'model.image_tower.image_tower.encoder.layers.9.layer_norm2.bias', 'model.image_tower.image_tower.encoder.layers.13.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.4.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.1.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.0.layer_norm1.weight', 'mo

ASSISTANT: In what location did I see the laptop?


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at LanguageBind/Video-LLaVA-7B were not used when initializing LlavaLlamaForCausalLM: ['model.video_tower.video_tower.encoder.layers.14.self_attn.out_proj.weight', 'model.video_tower.video_tower.encoder.layers.23.self_attn.q_proj.weight', 'model.video_tower.video_tower.encoder.layers.7.self_attn.q_proj.bias', 'model.video_tower.video_tower.encoder.layers.7.self_attn.out_proj.weight', 'model.image_tower.image_tower.encoder.layers.19.self_attn.k_proj.weight', 'model.image_tower.image_tower.encoder.layers.21.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.12.mlp.fc1.bias', 'model.image_tower.image_tower.encoder.layers.9.layer_norm2.bias', 'model.image_tower.image_tower.encoder.layers.13.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.4.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.1.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.0.layer_norm1.weight', 'mo

ASSISTANT: Where is the blue chair?


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at LanguageBind/Video-LLaVA-7B were not used when initializing LlavaLlamaForCausalLM: ['model.video_tower.video_tower.encoder.layers.14.self_attn.out_proj.weight', 'model.video_tower.video_tower.encoder.layers.23.self_attn.q_proj.weight', 'model.video_tower.video_tower.encoder.layers.7.self_attn.q_proj.bias', 'model.video_tower.video_tower.encoder.layers.7.self_attn.out_proj.weight', 'model.image_tower.image_tower.encoder.layers.19.self_attn.k_proj.weight', 'model.image_tower.image_tower.encoder.layers.21.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.12.mlp.fc1.bias', 'model.image_tower.image_tower.encoder.layers.9.layer_norm2.bias', 'model.image_tower.image_tower.encoder.layers.13.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.4.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.1.self_attn.v_proj.weight', 'model.image_tower.image_tower.encoder.layers.0.layer_norm1.weight', 'mo

ASSISTANT: In what spot did I see the bag?


In [ ]:
sample_50_videos_with_pred.to_csv(path_test_models + "sample_50_videos_with_pred_1_25.csv")